# Analizar partidos históricos Liga 1 2020-2024

In [2]:
import pandas as pd
from pathlib import Path
import re

In [334]:
df = pd.read_csv('data/2020 Alianza Universidad Stats, All Competitions | FBref.com.csv')

# Data Cleaning

In [335]:
df.drop(['Unnamed: 0', 'Match Report', 'Notes'], inplace=True, axis=1)
df.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,Poss,Attendance,Captain,Formation,Opp Formation,Referee
0,2020-01-31,20:30,Liga 1,Torneo Apertura,Fri,Away,W,3,2,Alianza Lima,NaN,NaN,Diego Hernán Morales,4-4-2,3-4-3,Augusto Menéndez
1,2020-02-09,15:00,Liga 1,Torneo Apertura,Sun,Home,W,1,0,Carlos Stein,NaN,NaN,NaN,4-2-3-1,5-3-2,David Huaman
2,2020-02-16,11:00,Liga 1,Torneo Apertura,Sun,Away,W,2,1,USMP,NaN,NaN,NaN,4-2-3-1,4-3-3,Jesús Cartagena
3,2020-02-24,20:00,Liga 1,Torneo Apertura,Mon,Home,W,1,0,Cienciano,NaN,NaN,NaN,4-3-3,4-4-2,Michael Espinoza
4,2020-02-29,20:00,Liga 1,Torneo Apertura,Sat,Home,W,3,0,Llacuabamba,NaN,NaN,NaN,NaN,NaN,Fernando Legario


## Revisar columnas

| Columnas | Descripción|
| -------- | ----------- |
| Date | Fecha del partido en formato año-mes-día|
| Time | Hora en formato 24h del partido. Tiempo local está en (-)|
| Comp | Competición que se disputó|
| Round | Ronda de la competición|
| Día | Día de la semana|
| Venue | Localidad del equipo|
| Result | W - Ganador, D - Empate, L - Derrota|
| GF | Goles a favor|
| GA | Goles en contra|
| Opponent | Openente del partido|
| Poss | Porcentaje del balón|
| Attendance | Asistencia de público|
| Captain | Capítan del equipo|
| Formation | Formación del equipo|
| Opp Formación | Formación del equipo rival|
| Referee | Árbitro del encuentro|
| Match Report | Reporte del partido|
| Notes | Notas del partido|
__Nota__: Cada tabla tiene datos de los partidos de un equipo

In [336]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           28 non-null     object 
 1   Time           28 non-null     object 
 2   Comp           28 non-null     object 
 3   Round          28 non-null     object 
 4   Day            28 non-null     object 
 5   Venue          28 non-null     object 
 6   Result         28 non-null     object 
 7   GF             28 non-null     int64  
 8   GA             28 non-null     int64  
 9   Opponent       28 non-null     object 
 10  Poss           0 non-null      float64
 11  Attendance     0 non-null      float64
 12  Captain        23 non-null     object 
 13  Formation      27 non-null     object 
 14  Opp Formation  27 non-null     object 
 15  Referee        28 non-null     object 
dtypes: float64(2), int64(2), object(12)
memory usage: 3.6+ KB


In [338]:
df['Team'] = 'Alianza Universidad'
df.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Team
0,2020-01-31,20:30,Liga 1,Torneo Apertura,Fri,Away,W,3,2,Alianza Lima,NaN,NaN,Diego Hernán Morales,4-4-2,3-4-3,Augusto Menéndez,Alianza Universidad
1,2020-02-09,15:00,Liga 1,Torneo Apertura,Sun,Home,W,1,0,Carlos Stein,NaN,NaN,NaN,4-2-3-1,5-3-2,David Huaman,Alianza Universidad
2,2020-02-16,11:00,Liga 1,Torneo Apertura,Sun,Away,W,2,1,USMP,NaN,NaN,NaN,4-2-3-1,4-3-3,Jesús Cartagena,Alianza Universidad
3,2020-02-24,20:00,Liga 1,Torneo Apertura,Mon,Home,W,1,0,Cienciano,NaN,NaN,NaN,4-3-3,4-4-2,Michael Espinoza,Alianza Universidad
4,2020-02-29,20:00,Liga 1,Torneo Apertura,Sat,Home,W,3,0,Llacuabamba,NaN,NaN,NaN,NaN,NaN,Fernando Legario,Alianza Universidad


## Parsear a datetime las columnas 'Hora' 'Fecha'

In [339]:
df['Date'].isnull().sum(), df['Time'].isnull().sum()

(np.int64(0), np.int64(0))

In [340]:
# Creando columna 'DateTime' con tipo de dato datetime64[ns]
fecha_hora = df['Date'] + ' ' + df['Time']
df['DateTime'] = pd.to_datetime(fecha_hora, format='%Y-%m-%d %H:%M')
df['DateTime'].head()                          

0   2020-01-31 20:30:00
1   2020-02-09 15:00:00
2   2020-02-16 11:00:00
3   2020-02-24 20:00:00
4   2020-02-29 20:00:00
Name: DateTime, dtype: datetime64[ns]

In [341]:
# Modificando columna 'Day'
df['Day'] = df['DateTime'].dt.dayofweek
df.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,Poss,Attendance,Captain,Formation,Opp Formation,Referee,Team,DateTime
0,2020-01-31,20:30,Liga 1,Torneo Apertura,4,Away,W,3,2,Alianza Lima,NaN,NaN,Diego Hernán Morales,4-4-2,3-4-3,Augusto Menéndez,Alianza Universidad,2020-01-31 20:30:00
1,2020-02-09,15:00,Liga 1,Torneo Apertura,6,Home,W,1,0,Carlos Stein,NaN,NaN,NaN,4-2-3-1,5-3-2,David Huaman,Alianza Universidad,2020-02-09 15:00:00
2,2020-02-16,11:00,Liga 1,Torneo Apertura,6,Away,W,2,1,USMP,NaN,NaN,NaN,4-2-3-1,4-3-3,Jesús Cartagena,Alianza Universidad,2020-02-16 11:00:00
3,2020-02-24,20:00,Liga 1,Torneo Apertura,0,Home,W,1,0,Cienciano,NaN,NaN,NaN,4-3-3,4-4-2,Michael Espinoza,Alianza Universidad,2020-02-24 20:00:00
4,2020-02-29,20:00,Liga 1,Torneo Apertura,5,Home,W,3,0,Llacuabamba,NaN,NaN,NaN,NaN,NaN,Fernando Legario,Alianza Universidad,2020-02-29 20:00:00


## Limpiando NaN

In [342]:
total = df.shape[0]
drop_columns = []
for col in df.columns:
    nan = df[col].isnull().sum()
    por = nan / total
    # Dropeamos las columnas con más de 70% de sus datos como NaN
    if (por > 0.7):
        drop_columns += [col]
    print(f'NaNs en la columna {col}: \t{nan}/{total}')
print(f'Columnas por eliminar: {drop_columns}')    

NaNs en la columna Date: 	0/28
NaNs en la columna Time: 	0/28
NaNs en la columna Comp: 	0/28
NaNs en la columna Round: 	0/28
NaNs en la columna Day: 	0/28
NaNs en la columna Venue: 	0/28
NaNs en la columna Result: 	0/28
NaNs en la columna GF: 	0/28
NaNs en la columna GA: 	0/28
NaNs en la columna Opponent: 	0/28
NaNs en la columna Poss: 	28/28
NaNs en la columna Attendance: 	28/28
NaNs en la columna Captain: 	5/28
NaNs en la columna Formation: 	1/28
NaNs en la columna Opp Formation: 	1/28
NaNs en la columna Referee: 	0/28
NaNs en la columna Team: 	0/28
NaNs en la columna DateTime: 	0/28
Columnas por eliminar: ['Poss', 'Attendance']


In [343]:
df.drop(drop_columns, axis=1, inplace=True)

In [344]:
# Revisamos la columna capitan
# Se puede imputar por frecuencia de valor
df['Captain'].unique()

array(['Diego Hernán Morales', nan], dtype=object)

In [345]:
# Revisamos la columna Formation
# Se puede imputar por frecuencia de valor
df['Formation'].unique()

array(['4-4-2', '4-2-3-1', '4-3-3', nan, '4-2-2-2'], dtype=object)

### Codificando Result

In [346]:
result = {'W': 0, 'D': 1, 'L': 2}
df['Result'] = df['Result'].replace(result)

/tmp/ipykernel_43373/894834085.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Result'] = df['Result'].replace(result)


## Listo para concatenar data

In [347]:
df.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,Captain,Formation,Opp Formation,Referee,Team,DateTime
0,2020-01-31,20:30,Liga 1,Torneo Apertura,4,Away,0,3,2,Alianza Lima,Diego Hernán Morales,4-4-2,3-4-3,Augusto Menéndez,Alianza Universidad,2020-01-31 20:30:00
1,2020-02-09,15:00,Liga 1,Torneo Apertura,6,Home,0,1,0,Carlos Stein,NaN,4-2-3-1,5-3-2,David Huaman,Alianza Universidad,2020-02-09 15:00:00
2,2020-02-16,11:00,Liga 1,Torneo Apertura,6,Away,0,2,1,USMP,NaN,4-2-3-1,4-3-3,Jesús Cartagena,Alianza Universidad,2020-02-16 11:00:00
3,2020-02-24,20:00,Liga 1,Torneo Apertura,0,Home,0,1,0,Cienciano,NaN,4-3-3,4-4-2,Michael Espinoza,Alianza Universidad,2020-02-24 20:00:00
4,2020-02-29,20:00,Liga 1,Torneo Apertura,5,Home,0,3,0,Llacuabamba,NaN,NaN,NaN,Fernando Legario,Alianza Universidad,2020-02-29 20:00:00


In [348]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           28 non-null     object        
 1   Time           28 non-null     object        
 2   Comp           28 non-null     object        
 3   Round          28 non-null     object        
 4   Day            28 non-null     int32         
 5   Venue          28 non-null     object        
 6   Result         28 non-null     int64         
 7   GF             28 non-null     int64         
 8   GA             28 non-null     int64         
 9   Opponent       28 non-null     object        
 10  Captain        23 non-null     object        
 11  Formation      27 non-null     object        
 12  Opp Formation  27 non-null     object        
 13  Referee        28 non-null     object        
 14  Team           28 non-null     object        
 15  DateTime       28 non-nul